# Project: Implementation of Gaussian discriminant analysis, Naive bayes and  Logistic regression

In [58]:
import pandas as pd
import numpy as np
import numpy.linalg as m
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [59]:
data=pd.read_csv('drugLibTrain_raw.tsv',delimiter='\t',encoding='utf-8')

In [60]:
data.head()

,Unnamed: 0,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,2202,enalapril,4,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ..."
1,3117,ortho-tri-cyclen,1,Highly Effective,Severe Side Effects,birth prevention,Although this type of birth control has more c...,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...","I Hate This Birth Control, I Would Not Suggest..."
2,1146,ponstel,10,Highly Effective,No Side Effects,menstrual cramps,I was used to having cramps so badly that they...,Heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...
3,3947,prilosec,3,Marginally Effective,Mild Side Effects,acid reflux,The acid reflux went away for a few months aft...,"Constipation, dry mouth and some mild dizzines...",I was given Prilosec prescription at a dose of...
4,1951,lyrica,2,Marginally Effective,Severe Side Effects,fibromyalgia,I think that the Lyrica was starting to help w...,I felt extremely drugged and dopey. Could not...,See above


In [61]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3107 entries, 0 to 3106
Data columns (total 9 columns):
Unnamed: 0           3107 non-null int64
urlDrugName          3107 non-null object
rating               3107 non-null int64
effectiveness        3107 non-null object
sideEffects          3107 non-null object
condition            3106 non-null object
benefitsReview       3107 non-null object
sideEffectsReview    3105 non-null object
commentsReview       3099 non-null object
dtypes: int64(2), object(7)
memory usage: 218.5+ KB


### Conversion of feature to binary numbers

In [62]:
#CountVectorizer(data['condition'],min_df=5,max_df=10,max_features=3107,binary=True)
#data=data.apply(lambda col: pd.factorize(col, sort=True)[0])
#data


count_vector= CountVectorizer(min_df=5,max_features=30,binary=True)
data['urlDrugName'] = data['urlDrugName'].apply(lambda x: str(x))
data['urlDrugName'] = data['urlDrugName'].str.replace('\d+', ' ' )
V= count_vector.fit_transform(data['urlDrugName'])
V= V.toarray()
data['urlDrugName']=V


data['rating']=data['rating'].apply(lambda x: 0 if x<5 else 1)


count_vector= CountVectorizer(min_df=5,max_features=30,binary=True)
data['condition'] = data['condition'].apply(lambda x: str(x))
data['condition'] = data['condition'].str.replace('\d+', ' ' )
X= count_vector.fit_transform(data['condition'])
X= X.toarray()
data['condition']=X

count_vector= CountVectorizer(min_df=5,max_features=30,binary=True)
data['benefitsReview'] = data['benefitsReview'].apply(lambda x: str(x))
data['benefitsReview'] = data['benefitsReview'].str.replace('\d+', ' ' )
Y= count_vector.fit_transform(data['benefitsReview'])
Y= Y.toarray()
data['benefitsReview'] = Y

count_vector= CountVectorizer(min_df=5,max_features=30,binary=True)
data['sideEffectsReview'] = data['sideEffectsReview'].apply(lambda x: str(x))
data['sideEffectsReview'] = data['sideEffectsReview'].str.replace('\d+', ' ' )
Z= count_vector.fit_transform(data['sideEffectsReview'])
Z= Z.toarray()
data['sideEffectsReview']=Z 

count_vector= CountVectorizer(min_df=5,max_features=30,binary=True)
data['commentsReview'] = data['commentsReview'].apply(lambda x: str(x))
data['commentsReview'] = data['commentsReview'].str.replace('\d+', ' ' )
T= count_vector.fit_transform(data['commentsReview'])
T= T.toarray()
data['commentsReview']=T

np.sum(X==1,axis=0)
D= data[['effectiveness','sideEffects']]
D=pd.get_dummies(D)
D=pd.DataFrame(D)
D

,effectiveness_Considerably Effective,effectiveness_Highly Effective,effectiveness_Ineffective,effectiveness_Marginally Effective,effectiveness_Moderately Effective,sideEffects_Extremely Severe Side Effects,sideEffects_Mild Side Effects,sideEffects_Moderate Side Effects,sideEffects_No Side Effects,sideEffects_Severe Side Effects
0,0,1,0,0,0,0,1,0,0,0
1,0,1,0,0,0,0,0,0,0,1
2,0,1,0,0,0,0,0,0,1,0
3,0,0,0,1,0,0,1,0,0,0
4,0,0,0,1,0,0,0,0,0,1
5,0,0,1,0,0,0,0,0,0,1
6,0,1,0,0,0,0,1,0,0,0
7,1,0,0,0,0,0,0,0,1,0
8,0,1,0,0,0,0,0,0,1,0
9,0,0,1,0,0,1,0,0,0,0


In [63]:
data.head()

,Unnamed: 0,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,2202,0,0,Highly Effective,Mild Side Effects,0,0,0,0
1,3117,0,0,Highly Effective,Severe Side Effects,0,0,0,0
2,1146,0,1,Highly Effective,No Side Effects,0,0,0,0
3,3947,0,0,Marginally Effective,Mild Side Effects,1,1,1,0
4,1951,0,0,Marginally Effective,Severe Side Effects,0,0,0,0


In [64]:
da  = data.pop('Unnamed: 0')
da1 = data.pop('effectiveness')
da2 = data.pop('sideEffects')

In [65]:
data

,urlDrugName,rating,condition,benefitsReview,sideEffectsReview,commentsReview
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,1,0,0,0,0
3,0,0,1,1,1,0
4,0,0,0,0,0,0
5,0,0,0,1,0,0
6,0,1,0,0,0,0
7,0,1,0,0,0,0
8,0,1,0,1,1,0
9,0,0,0,0,0,0


In [66]:
d=data.join(D)

In [67]:
d.head()

,urlDrugName,rating,condition,benefitsReview,sideEffectsReview,commentsReview,effectiveness_Considerably Effective,effectiveness_Highly Effective,effectiveness_Ineffective,effectiveness_Marginally Effective,effectiveness_Moderately Effective,sideEffects_Extremely Severe Side Effects,sideEffects_Mild Side Effects,sideEffects_Moderate Side Effects,sideEffects_No Side Effects,sideEffects_Severe Side Effects
0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
2,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0
3,0,0,1,1,1,0,0,0,0,1,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1


In [11]:
d.shape

(3107, 16)

### Calculus of Gaussian discriminant analysis parameters 

In [12]:
def calculate_phi(Y):
    m = length(Y)
    return sum(x(x==1), Y)/m

def calculate_sigma(X, Y, mu0, mu1):
    m = length(Y)
    sqr_sum = zeros(size(X)[2], size(X)[2])
    for i in range(m):
        xi = X[i,:]
        yi = Y[i]
        sqr = sum((xi-mu1)*(xi-mu1).T(yi == 1)) 
        sqr_sum = sqr_sum + sqr
 
    return sqr_sum/m


def calculate_mu1(X, Y):
    m = length(Y)
    y_pos = count(x(x==1), Y)
    conditional_sum_x = zeros(X[1,:])
    for i in range(m):
        xi = X[i,:]
        yi = Y[i]
        conditional_sum_x = conditional_sum_x + sum(xi(yi == 1), zeros(xi))
    return (1/m)*conditional_sum_x/y_pos


def calculate_mu0(X, Y):
    m = length(Y)
    y_neg = count(x(x==0), Y)
    conditional_sum_x = zeros(X[1,:])
    for i in range(m):
        xi = X[i,:]
        yi = Y[i]
        conditional_sum_x = conditional_sum_x + sum(xi(yi == 0), zeros(xi))
    return (1/m)*conditional_sum_x/y_neg

def calculate_px_py(x, mu, sigma):
    n = 1
    pi = 3.14
    dim = len(mu)
    return ((1/(2*np.pi)^(dim/2)*np.sqrt(m.det(sigma)))*np.exp(-0.5*np.transpose(x-mu)*np.linalg.inv(sigma)*(x-mu)))

                                                  
def calculate_py(y, phi):
    return sum(phi(y==1), (1-phi))

def predict(self, X):
        y_pred = []
        for sample in X:
            h = sample.dot(self.w)
            y = 1 * (h < 0)
            y_pred.append(y)
        return y_pred


In [13]:
def cov(x, y):
    xbar, ybar = x.mean(), y.mean()
    return np.sum((x - xbar)*(y - ybar))/(len(x) - 1)

def calculate_covariance_matrix(X):
    return np.array([[cov(X[0], X[0]), cov(X[0], X[1])], \
                     [cov(X[1], X[0]), cov(X[1], X[1])]])

### Gaussian discriminant analysis class

In [14]:
class Gaussian_Discriminant_Analysis():
    
    def __init__(self):
        self.w = None

    def transform(self, X, y):
        self.fit(X, y)
        # Project data onto vector
        X_transform = X.dot(self.w)
        return X_transform

    def fit(self, X, y):
        # Separate data by class
        X1 = X[y == 0]
        X2 = X[y == 1]

        # Calculate the covariance matrices of the two datasets
        cov1 = calculate_covariance_matrix(X1)
        cov2 = calculate_covariance_matrix(X2)
        cov_tot = cov1 + cov2

        # Calculate the mean of the two datasets
        mean1 = X1.mean(0)
        mean2 = X2.mean(0)
        mean_diff = np.atleast_1d(mean1 - mean2) #convert to an array to 1D at least

        # Determine the vector which when X is projected onto it best separates the
        # data by class. w = (mean1 - mean2) / (cov1 + cov2)
        self.w = np.linalg.pinv(cov_tot).dot(mean_diff)

    def predict(self, X):
        y_pred = []
        for sample in X:
            h = sample.dot(self.w)
            y = 1 * (h < 0)
            y_pred.append(y)
        return y_pred
    
    def accuracy(self,X,y):
        y_pred = self.predict(X)
        accu=np.sum(y_pred == y)/len(y)
        return accu

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

data = np.genfromtxt("./gaussian_discriminant_analysis.csv", dtype=np.float64, delimiter=",")
fig = plt.figure()
#ax = Axes3D(fig)

class GDA():
    """
    Gaussian Discriminant Analysis
    """

    def __init__(self):
        self.phi = None
        self.mu0 = None
        self.mu1 = None
        self.sigma = None

    def train(self, x, y):
        """
        """
        self.phi = np.mean(y)
        self.mu0 = np.mean(x[y[:,0]==0], axis=0)
        self.mu1 = np.mean(x[y[:,0]==1], axis=0)

        n_x = x[y[:,0]==0] - self.mu0
        p_x = x[y[:,0]==1] - self.mu1

        self.sigma = ((n_x.T).dot(n_x) + (p_x.T).dot(p_x))/x.shape[0]

if __name__ == "__main__":
    trainX = data[:,:2]
    trainY = data[:,-1:]

    gda = GDA()
    gda.train(trainX, trainY)

    pos_data = data[data[:,-1]==1]
    neg_data = data[data[:,-1]==0]
    plt.scatter(pos_data[:,0], pos_data[:,1], color="red")
    plt.scatter(neg_data[:,0], neg_data[:,1], color="blue")

    phi = gda.phi
    sigma = gda.sigma
    inv_sigma = np.linalg.inv(sigma)
    mu0 = gda.mu0.reshape(-1, 1)
    mu1 = gda.mu1.reshape(-1, 1)

    """
    Draw GDA Decision Boundary
    """
    k = (2 * inv_sigma).dot(mu1-mu0)
    b = (mu1.T).dot(inv_sigma).dot(mu1) - (mu0.T).dot(inv_sigma).dot(mu0) + np.log(phi) - np.log(1-phi)

    x1 = np.arange(-1, 10, 1)
    x2 = (b[0][0] - k[0][0]*x1)/k[1][0]

    """
    Draw contour
    """
    xgrid1 = np.arange(-2, 4, 0.1)
    ygrid1 = np.arange(-1, 6, 0.1)
    xgrid0 = np.arange(4, 10, 0.1)
    ygrid0 = np.arange(5, 12, 0.1)
    X1, Y1 = np.meshgrid(xgrid1, ygrid1)
    X0, Y0 = np.meshgrid(xgrid0, ygrid0)

    Z1 = np.exp(-0.35*((X1-1)**2+0.85*(Y1-2)**2)) # Compute function values on the grid
    Z0 = np.exp(-0.35*((X0-7)**2+0.85*(Y0-8)**2))
    CS1 = plt.contour(X1,Y1,Z1)
    CS0 = plt.contour(X0,Y0,Z0)

    plt.plot(x1,x2)
plt.show()

In [15]:
d.columns

Index(['urlDrugName', 'rating', 'condition', 'benefitsReview',
       'sideEffectsReview', 'commentsReview',
       'effectiveness_Considerably Effective',
       'effectiveness_Highly Effective', 'effectiveness_Ineffective',
       'effectiveness_Marginally Effective',
       'effectiveness_Moderately Effective',
       'sideEffects_Extremely Severe Side Effects',
       'sideEffects_Mild Side Effects', 'sideEffects_Moderate Side Effects',
       'sideEffects_No Side Effects', 'sideEffects_Severe Side Effects'],
      dtype='object')

### Test of Gaussian discriminant analysis on iris data

In [16]:
data=datasets.load_iris()
x1=data.data[:,:2]
y1=(data.target!=0)*1

In [17]:
model=Gaussian_Discriminant_Analysis()

In [18]:
model.fit(x1,y1)

In [19]:
predictions=model.predict(x1)

In [20]:
model.accuracy(x1,y1)

0.6666666666666666

### Test of Gaussian discriminant analysis on a given data

In [22]:
model1=Gaussian_Discriminant_Analysis()

In [27]:
predictions=model1.predict(x1)

In [29]:
model1.accuracy(x1,y1)

0.6666666666666666

### Implementation of Naive Bayes 

In [ ]:
 
from csv import reader
from random import seed
from random import randrange
from math import sqrt
from math import exp
from math import pi
from sklearn import datasets
 
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset
 
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
        
# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup
 
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
            predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores
 
# Split the dataset by class values, returns a dictionary
def separate_by_class(dataset):
    separated = dict()
    for i in range(len(dataset)):
        vector = dataset[i]
        class_value = vector[-1]
        if (class_value not in separated):
            separated[class_value] = list()
            separated[class_value].append(vector)
    return separated
 
# Calculate the mean of a list of numbers
def mean(numbers):
    return sum(numbers)/float(len(numbers))
 
# Calculate the standard deviation of a list of numbers
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([(x-avg)**2 for x in numbers]) / float(len(numbers)-1)
    return sqrt(variance)
 
# Calculate the mean, stdev and count for each column in a dataset
def summarize_dataset(dataset):
    summaries = [(mean(column), stdev(column), len(column)) for column in zip(*dataset)]
    del(summaries[-1])
    return summaries
 
# Split dataset by class then calculate statistics for each row
def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    summaries = dict()
    for class_value, rows in separated.items():
        summaries[class_value] = summarize_dataset(rows)
    return summaries
 
# Calculate the Gaussian probability distribution function for x
def calculate_probability(x, mean, stdev):
    exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent
 
# Calculate the probabilities of predicting each class for a given row
def calculate_class_probabilities(summaries, row):
    total_rows = sum([summaries[label][0][2] for label in summaries])
    probabilities = dict()
    for class_value, class_summaries in summaries.items():
        probabilities[class_value] = summaries[class_value][0][2]/float(total_rows)
        for i in range(len(class_summaries)):
            mean, stdev, _ = class_summaries[i]
            probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
    return probabilities
 
# Predict the class for a given row
def predict(summaries, row):
    probabilities = calculate_class_probabilities(summaries, row)
    best_label, best_prob = None, -1
    for class_value, probability in probabilities.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value
    return best_label
 
# Naive Bayes Algorithm
def naive_bayes(train, test):
    summarize = summarize_by_class(train)
    predictions = list()
    for row in test:
        output = predict(summarize, row)
        predictions.append(output)
    return(predictions)

In [39]:
class Naive_bayes:
    def __init__(self,X,y):
        self.X = X
        self.y = y
    def fit(self):
        data = self.X
        target = self.y
        x_1y_0=[]
        x_1y_1=[]
        y_new=[]
        count_3=0
        count_4=0
        for j in range(data.shape[1]):
            count=0
            count_2=0
            for i in range(data.shape[0]):
#                 print(data,target,i,j)
                if y[i]==0 and data[i][j]==1:
                    count +=1
                if y[i]==1 and data[i][j]==1:
                    count_2 +=1
                if j==0 and y[i]==1:
                    count_3+=1
                if j==0 and y[i]==0:
                    count_4+=1
            length_1=count_3
            length_0=count_4
            x_1y_0.append(count/length_0)
            x_1y_1.append(count_2/length_1)
        y_new.append(count_3/(length_0+length_1))
        return x_1y_0,x_1y_1,y_new

    def Bernouilli(self,x,phi):
        return (phi**x)*(1-phi)**(1-x)

    def predict(self,x):
        phi_x1_y0,phi_x1_y1,p = self.fit()
        pred = np.zeros(x.shape[0])
        for i in range(len(x)):
            prb1 = 1
            prb2 = 1
            for j in range(x.shape[1]):
                prb1*= self.Bernouilli(x[i,j],phi_x1_y0[j])
                prb2*= self.Bernouilli(x[i,j],phi_x1_y1[j])
            prb1 = prb1*(1-p[0])
            prb2 = prb2*(p[0])
            pred[i] = np.argmax([prb1,prb2])
        return pred

    def accuracy(self,test,pred):
        c=0
        for i in range(len(pred)):
            if(test.iloc[i]==pred[i]):
                c+=1
        c=c/len(pred)
        return c

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=101)

### Test Naive Bayes on Iris Dataset

In [ ]:
#naive_bayes(x1,y1)

In [ ]:
#seed(1)
#filename = 'iris.csv'
#dataset = load_csv(filename)
#for i in range(len(x1)):
 #   str_column_to_float(x1, i)
# convert class column to integers
#str_column_to_int(x1, len(x1))
# evaluate algorithm
#n_folds = 5
#scores = evaluate_algorithm(x, naive_bayes, n_folds)
#print('Scores: %s' % scores)
#print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

In [ ]:
#seed(1)
#dataset=datasets.load_iris()
#filename = 'iris.csv'
#dataset = load_csv(filename)
#for i in range(len(dataset[data])):
 #   str_column_to_float(dataset, i)
# convert class column to integers
#str_column_to_int(dataset, len(dataset['data'])-1)
# evaluate algorithm
#n_folds = 5
#scores = evaluate_algorithm(dataset, naive_bayes, n_folds)
#print('Scores: %s' % scores)
#print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

### Test Naive Bayes on a given Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=101)

In [ ]:
m=Naive_bayes(X_train,y_train)

In [ ]:
m.fit()

### Logistic regression class

In [ ]:
class LogisticRegression():
    def __init__(self,lr,n_iters,weight,x,y):
        self.lr = lr
        self.n_iters = n_iters
        self.weight = weight
        self.x = x
        self.y = y
        
    def Sigmoid(self,z):
        sign = 1 / (1+np.exp(-z))
        return sign
    
    def Cost(self):
        yhat= self.x@self.weight
        cost = ((-self.y.T) @ np.log2(self.Sigmoid(yhat))) - ((1 - self.y).T @ np.log2(1 - self.Sigmoid(yhat)))
        #print(yhat.shape)
        return cost
    
    def Gradient(self):
        yhat= self.x@self.weight
        return self.x.T @ ((self.y) - self.Sigmoid(yhat))
    
    def fit(self):
        loss = np.zeros((self.n_iters))
        for i in range(self.n_iters):
            self.weight = self.weight + (self.lr * self.Gradient())
            
            loss[i] = self.Cost()
            
        return self.weight, loss
    
    
    def pred_prob(self,z):
            return self.Sigmoid(np.dot(z,self.weight))
    
    def predict(self,prediction):
    
         liste = []

         for elt in prediction:
                if elt<0.5:
                    liste.append(0)
                else:
                    liste.append(1)
                liste = np.array(liste)
                liste = liste.reshape(-1,1)
                return liste
    
    def hessian(self):
        yhat = self.x@self.weight
        omega = self.Sigmoid(yhat) @ (1 - self.Sigmoid(yhat)).T
        return ((-x.T) @ omega)@x -2.44641553e+05, LA.eigvals(((-x.T) @ omega)@x)

    def Newton(self):
        for i in range(self.n_iters):
            self.weight = self.weight - inv(self.hessian()[0])@self.Gradient()
        return self.weight
    
    def Good_pred(self,z):
        
        yhat=z@self.weight
        
        return np.where(self.Sigmoid(yhat)>0.5,1,0)
    
    
def accuracy(prediction,y):
    acc = y==prediction
    return np.sum(acc)/len(y)
        
def Sigmoid(x):
    sign = 1 / (1+np.exp(-x))
    return sign

def Good_pred(z, weight):
        
        yhat=z@weight
        
        return np.where(Sigmoid(yhat)>0.5,1,0)        

### Test of logistic regression on iris data

In [ ]:
data=datasets.load_iris()
x1=data.data[:,:2]
y1=(data.target!=0)*1
y1=y1.reshape(-1,1)
theta = np.zeros((x1.shape[1],1))       

In [ ]:
model=LogisticRegression(0.01,10000,theta,x1,y1)

In [ ]:
result=model.fit()

In [ ]:
pred1= Good_pred(x1,result[0])
theta = np.zeros((x1.shape[1],1))

In [ ]:
accuracy(pred1,y1)

### Test of logistic regression on a given data

### Convertion of data syntax

In [ ]:
#convert all objects to categories
#object_types = data.select_dtypes(include=['O'])
#for col in object_types:
 #   data['{0}_category'.format(col)] = data[col].astype('category')

In [ ]:
#convert all objects to int values
#object_types = dataset.select_dtypes(include=['O'])

#new_cols = {}
#for col in object_types:
 #   data_set = set(dataset[col].tolist())
  #  data_indexed = {}
   # for i, item in enumerate(data_set):
    #    data_indexed[item] = i
    #new_list = []
    #for item in dataset[col].tolist():
     #   new_list.append(data_indexed[item])
    #new_cols[col]=new_list

#for key, val in new_cols.items():
 #   dataset['{0}_int_value'.format(key)] = val

In [ ]:
# x.apply(lambda col: pd.factorize(col)[0])

In [ ]:
#data=data.apply(lambda s: s.map({k:i for i,k in enumerate(s.unique())}))

### Thank you!